Stuff like Runners, Statistics, logging to tensorboard, saving and loading models

In [1]:
import os 
import tensorflow as tf

import numpy as np

from energy_py import ensure_dir

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


# Saver and loader

In [2]:
class Saver(object):
    """
    Inspired by deep-rl-tensorflow Statistic object
    
    Would want to give this model a list of the variables for acting (ie the online net), 
    or could do more ifwanted to learn later
    """
    
    def __init__(self, sess, model_dir, variables):
        self.sess = sess
        self.model_dir = model_dir
                                                                              
        ensure_dir(self.model_dir)
                
        self.step = tf.Variable(1, name='saver_step', dtype=tf.int32)
        self.increase_step = self.step.assign_add(1)
        
        variables = variables + [self.step]
        self.saver = tf.train.Saver(variables, max_to_keep=20)
        print(variables)

    def save_model(self):
        """
        Saves the variables
        
        tf.train.global_step = the number of batches seen by the graph - not what we want
        """
        print('Saving model checkpoint')
        self.saver.save(self.sess, self.model_dir, global_step=self.step)
        self.sess.run(self.increase_step)
        
    def load_model(self):
        """
        Loads variables from a model checkpoint in model_dir
        """
        checkpoint = tf.train.get_checkpoint_state(self.model_dir)
        
        if checkpoint and checkpoint.model_checkpoint_path:
            checkpoint_name = os.path.basename(checkpoint.model_checkpoint_path)
            
            filename = os.path.join(self.model_dir, checkpoint_name)
            
            self.saver.restore(self.sess, filename)
            print('Loaded model checkpoint {}'.format(filename))

In [3]:
def make_graph(scope):
    with tf.variable_scope(scope):
        x = tf.placeholder(shape=(None), dtype=tf.float32)
        y = tf.placeholder(shape=(None), dtype=tf.float32)
        out = tf.multiply(x, y)
    
    return x, y, out

x, y, out = make_graph('saver')

with tf.Session() as sess:
    
    variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='saver')
    s = Saver(sess, './model_dir/', variables)
    sess.run(tf.global_variables_initializer())
    
    for step in range(3):
        save_step, o = sess.run([s.step, out], {x: step, y: step})
        print('save_step {} out {}'.format(save_step, o))
        
        s.save_model()
        

[<tf.Variable 'saver_step:0' shape=() dtype=int32_ref>]
save_step 1 out 0.0
Saving model checkpoint
save_step 2 out 1.0
Saving model checkpoint
save_step 3 out 4.0
Saving model checkpoint


In [4]:
tf.reset_default_graph()
x, y, out = make_graph('saver')

with tf.Session() as new_sess:
    
    variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='saver')
    new_s = Saver(new_sess, './model_dir/', variables)
    
    new_s.load_model()
    
    loaded_step = new_sess.run(new_s.step)

[<tf.Variable 'saver_step:0' shape=() dtype=int32_ref>]
INFO:tensorflow:Restoring parameters from ./model_dir/-3
Loaded model checkpoint ./model_dir/-3


In [5]:
loaded_step

3

# Experiment statistics

In [6]:
from energy_py import ensure_dir


class ExperimentStatsMan(object):
    
    def __init__(self,
                 sess, 
                 tb_dir):
        
        self.sess = 
        
        ensure_dir(tb_dir)
        self.writer = tf.summary.FileWriter(tb_dir, self.sess.graph)
        
        self.reset()
        
    def reset(self):
        self.episode_rewards = []
        self.current_episode_rewards = [] 
        
    def record_step(self, step, reward):
        print('Recording step {}'.format(step))
        
        self.current_episode_rewards.append(reward)
        
    def record_episode(self):
        
        total_episode_reward = sum(self.current_episode_rewards)
        self.episode_rewards.append(total_episode_reward)
        
        print('Recorded episode {}'.format(len(self.episode_rewards)))
        
        summaries = {
            'total_episode_reward': total_episode_reward,
            'avg_rew': np.mean(self.episode_rewards[-50:]),
            'min_rew': np.min(self.episode_rewards[-50:]),
            'max_rew': np.max(self.episode_rewards[-50:])
        }
        
        for tag, value in summmaries:
            summ = tf.Summary(
                value=[tf.Summary.Value(tag=tag, value=float(value))]
            )
            self.writer.add_summary(summary, len(self.episode_rewards))
        S
        self.current_episode_rewards = []

In [7]:
with tf.Session() as sess:
    e = ExperimentStatsMan(sess, './tb_dir')
    
    for step in range(3):
        e.record_step(step, step * 3)
    e.record_episode()
    
    for step in range(9):
        e.record_step(step, step * 3)
    e.record_episode()

Recording step 0
Recording step 1
Recording step 2
Recorded episode 1
Recording step 0
Recording step 1
Recording step 2
Recording step 3
Recording step 4
Recording step 5
Recording step 6
Recording step 7
Recording step 8
Recorded episode 2


# Summarizer

Wraps around tensorboard stuff

In [ ]:
def Summarizer(object):
    
    def __init__(self, sess, summaries, log_dir=None):
        summaries = summaries
        self.summary_op = tf.summary.merge(inputs=summaries)
        
        self.summarizer = tf.summary.FileWriter(
            logdir=None,
            graph=self.sess.graph,
            )
    
    def write_summaries(self, summaries):
        